In [1]:
from google.colab import files
uploaded = files.upload()

Saving d.csv to d.csv


In [2]:
import io
import pandas as pd
df = pd.read_csv(io.BytesIO(uploaded['d.csv']))
# Dataset is now stored in a Pandas Dataframe

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
plt.style.use('fivethirtyeight') # For plots

In [4]:
df

,DateTime,volume
0,2017-04-01 00:00:00,10
1,2017-04-01 01:00:00,12
2,2017-04-01 02:00:00,5
3,2017-04-01 03:00:00,5
4,2017-04-01 04:00:00,17
...,...,...
11563,2018-07-31 19:00:00,545
11564,2018-07-31 20:00:00,874
11565,2018-07-31 21:00:00,386
11566,2018-07-31 22:00:00,223


In [5]:
df['volume']=np.log(df['volume'])

In [6]:
#Adding features like hour, end_of_month, start_of_month, Weekday

df.DateTime = pd.to_datetime(df.DateTime)
df["hour"] = df.DateTime.dt.hour
#df["month"] = df.DateTime.dt.month
#df["week"] = df.DateTime.dt.week
#df["day"] = df.DateTime.dt.day
#df["dayofweek"] = df.DateTime.dt.dayofweek
df['end_of_month']=df.DateTime.dt.is_month_end.astype(int)
df['start_of_month']=df.DateTime.dt.is_month_start.astype(int)
df['WEEKDAY'] = ((pd.DatetimeIndex(df['DateTime']).dayofweek)).astype(int)

df

,DateTime,volume,hour,end_of_month,start_of_month,WEEKDAY
0,2017-04-01 00:00:00,2.302585,0,0,1,5
1,2017-04-01 01:00:00,2.484907,1,0,1,5
2,2017-04-01 02:00:00,1.609438,2,0,1,5
3,2017-04-01 03:00:00,1.609438,3,0,1,5
4,2017-04-01 04:00:00,2.833213,4,0,1,5
...,...,...,...,...,...,...
11563,2018-07-31 19:00:00,6.300786,19,1,0,1
11564,2018-07-31 20:00:00,6.773080,20,1,0,1
11565,2018-07-31 21:00:00,5.955837,21,1,0,1
11566,2018-07-31 22:00:00,5.407172,22,1,0,1


In [7]:
train=df.iloc[:10826]
train

,DateTime,volume,hour,end_of_month,start_of_month,WEEKDAY
0,2017-04-01 00:00:00,2.302585,0,0,1,5
1,2017-04-01 01:00:00,2.484907,1,0,1,5
2,2017-04-01 02:00:00,1.609438,2,0,1,5
3,2017-04-01 03:00:00,1.609438,3,0,1,5
4,2017-04-01 04:00:00,2.833213,4,0,1,5
...,...,...,...,...,...,...
10821,2018-06-30 19:00:00,6.590301,19,1,0,5
10822,2018-06-30 20:00:00,6.694562,20,1,0,5
10823,2018-06-30 21:00:00,5.940171,21,1,0,5
10824,2018-06-30 22:00:00,5.298317,22,1,0,5


In [8]:
test=df.iloc[10826:]
test

,DateTime,volume,hour,end_of_month,start_of_month,WEEKDAY
10826,2018-07-01 00:00:00,3.912023,0,0,1,6
10827,2018-07-01 01:00:00,3.367296,1,0,1,6
10828,2018-07-01 02:00:00,1.386294,2,0,1,6
10829,2018-07-01 03:00:00,0.693147,3,0,1,6
10830,2018-07-01 04:00:00,3.091042,4,0,1,6
...,...,...,...,...,...,...
11563,2018-07-31 19:00:00,6.300786,19,1,0,1
11564,2018-07-31 20:00:00,6.773080,20,1,0,1
11565,2018-07-31 21:00:00,5.955837,21,1,0,1
11566,2018-07-31 22:00:00,5.407172,22,1,0,1


In [10]:
#Creating train dataset

train_data=pd.DataFrame()
train_data['ds']=train['DateTime']
train_data['y']=train['volume']
train_data['hour']=train['hour']
train_data['end_of_month']=train['end_of_month']
train_data['start_of_month']=train['start_of_month']
train_data['WEEKDAY']=train['WEEKDAY']
train_data

,ds,y,hour,end_of_month,start_of_month,WEEKDAY
0,2017-04-01 00:00:00,2.302585,0,0,1,5
1,2017-04-01 01:00:00,2.484907,1,0,1,5
2,2017-04-01 02:00:00,1.609438,2,0,1,5
3,2017-04-01 03:00:00,1.609438,3,0,1,5
4,2017-04-01 04:00:00,2.833213,4,0,1,5
...,...,...,...,...,...,...
10821,2018-06-30 19:00:00,6.590301,19,1,0,5
10822,2018-06-30 20:00:00,6.694562,20,1,0,5
10823,2018-06-30 21:00:00,5.940171,21,1,0,5
10824,2018-06-30 22:00:00,5.298317,22,1,0,5


In [11]:
#Creating test dataset

test_data=pd.DataFrame()
test_data['ds']=test['DateTime']
test_data['y']=test['volume']
test_data['hour']=test['hour']
test_data['end_of_month']=test['end_of_month']
test_data['start_of_month']=test['start_of_month']
test_data['WEEKDAY']=test['WEEKDAY']
test_data

,ds,y,hour,end_of_month,start_of_month,WEEKDAY
10826,2018-07-01 00:00:00,3.912023,0,0,1,6
10827,2018-07-01 01:00:00,3.367296,1,0,1,6
10828,2018-07-01 02:00:00,1.386294,2,0,1,6
10829,2018-07-01 03:00:00,0.693147,3,0,1,6
10830,2018-07-01 04:00:00,3.091042,4,0,1,6
...,...,...,...,...,...,...
11563,2018-07-31 19:00:00,6.300786,19,1,0,1
11564,2018-07-31 20:00:00,6.773080,20,1,0,1
11565,2018-07-31 21:00:00,5.955837,21,1,0,1
11566,2018-07-31 22:00:00,5.407172,22,1,0,1


In [13]:
#Building model and adding Holiday Feature

m=Prophet(interval_width=0.95,yearly_seasonality='auto',weekly_seasonality='auto',daily_seasonality='auto')
m.add_country_holidays(country_name='IN')

In [14]:
#Adding features

m.add_regressor('hour',standardize=False)
m.add_regressor('end_of_month',standardize=False)
m.add_regressor('start_of_month',standardize=False)
m.add_regressor('WEEKDAY',standardize=False)


In [15]:
m.fit(train_data)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
/usr/local/lib/python3.6/dist-packages/fbprophet/hdays.py:259: Warning:

We only support Diwali and Holi holidays from 2010 to 2030



In [16]:
m.params

{'beta': array([[ 9.10184164e-03,  9.38326257e-03, -6.15865408e-03,
         -1.12458067e-02,  4.23220999e-03,  4.88658725e-03,
         -1.58953265e-01, -2.81522517e-01, -8.94546548e-02,
         -3.73761443e-02,  1.98631121e-03,  9.12387622e-03,
          1.24127870e-04,  8.68608867e-03, -1.46582481e-02,
          5.08232302e-02, -3.96078814e-02, -4.48607875e-02,
         -2.61354302e-02, -2.59103117e-02, -1.12328183e-02,
         -2.46521336e-02,  1.64240558e-02,  1.23079924e-02,
          4.14759838e-02,  3.82359221e-02,  1.78455438e-02,
         -6.07400673e-02, -3.00368050e-02, -1.41775704e-02,
         -4.98667665e-02, -5.36956119e-03, -3.19621011e-02,
         -2.02312881e-02, -1.21830641e-02, -4.17386812e-02,
         -5.98776619e-04,  7.84770899e-03,  4.32464253e-02,
         -5.60226594e-03]]),
 'delta': array([[ 3.37332053e-02,  6.72544258e-02,  3.58120887e-02,
          5.18046528e-02,  3.18321769e-05, -3.92855048e-03,
         -4.84773210e-04,  5.34862230e-04,  1.10492988

In [18]:
#Making future DataFrame

future=m.make_future_dataframe(periods=744,freq='H',include_history=False)
future

,ds
0,2018-07-01 00:00:00
1,2018-07-01 01:00:00
2,2018-07-01 02:00:00
3,2018-07-01 03:00:00
4,2018-07-01 04:00:00
...,...
739,2018-07-31 19:00:00
740,2018-07-31 20:00:00
741,2018-07-31 21:00:00
742,2018-07-31 22:00:00


In [20]:
#Making features for future DataFrame

future['hour']=future.ds.dt.hour
future["end_of_month"] =future.ds.dt.is_month_end.astype(int)
future["start_of_month"] = future.ds.dt.is_month_start.astype(int)
future['WEEKDAY'] = ((pd.DatetimeIndex(future['ds']).dayofweek)).astype(int)
future

,ds,hour,end_of_month,start_of_month,WEEKDAY
0,2018-07-01 00:00:00,0,0,1,6
1,2018-07-01 01:00:00,1,0,1,6
2,2018-07-01 02:00:00,2,0,1,6
3,2018-07-01 03:00:00,3,0,1,6
4,2018-07-01 04:00:00,4,0,1,6
...,...,...,...,...,...
739,2018-07-31 19:00:00,19,1,0,1
740,2018-07-31 20:00:00,20,1,0,1
741,2018-07-31 21:00:00,21,1,0,1
742,2018-07-31 22:00:00,22,1,0,1


In [21]:
#Predicting volume

forecast=m.predict(future)
forecast[['ds','yhat','yhat_lower','yhat_upper']]

/usr/local/lib/python3.6/dist-packages/fbprophet/hdays.py:259: Warning:

We only support Diwali and Holi holidays from 2010 to 2030

/usr/local/lib/python3.6/dist-packages/fbprophet/hdays.py:259: Warning:

We only support Diwali and Holi holidays from 2010 to 2030



,ds,yhat,yhat_lower,yhat_upper
0,2018-07-01 00:00:00,2.789964,1.811225,3.670489
1,2018-07-01 01:00:00,2.160438,1.190519,3.131569
2,2018-07-01 02:00:00,1.791975,0.896798,2.735563
3,2018-07-01 03:00:00,1.826798,0.948200,2.725553
4,2018-07-01 04:00:00,2.312337,1.412818,3.322258
...,...,...,...,...
739,2018-07-31 19:00:00,6.339043,5.428193,7.226217
740,2018-07-31 20:00:00,5.716208,4.821205,6.747003
741,2018-07-31 21:00:00,4.966792,4.050402,5.891814
742,2018-07-31 22:00:00,4.179627,3.228780,5.073771


In [22]:
#Making use of exp function for converting log values to regular

forecast['yhat']=np.exp(forecast['yhat'])
forecast['yhat_lower']=np.exp(forecast['yhat_lower'])
forecast['yhat_upper']=np.exp(forecast['yhat_upper'])
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,All Saints Day,All Saints Day_lower,All Saints Day_upper,All Souls Day,All Souls Day_lower,All Souls Day_upper,Boxing Day,Boxing Day_lower,Boxing Day_upper,Christmas Day,Christmas Day_lower,Christmas Day_upper,Day of Ashura,Day of Ashura_lower,Day of Ashura_upper,Diwali,Diwali_lower,Diwali_upper,Easter Sunday,Easter Sunday_lower,Easter Sunday_upper,Eid al-Fitr,Eid al-Fitr_lower,Eid al-Fitr_upper,Feast of Holy Family,Feast of Holy Family_lower,Feast of Holy Family_upper,Feast of Pentecost,Feast of Pentecost_lower,Feast of Pentecost_upper,Feast of the Blessed Virgin,Feast of the Blessed Virgin_lower,Feast of the Blessed Virgin_upper,Feast of the Sacrifice,...,New Year's Day,New Year's Day_lower,New Year's Day_upper,Palm Sunday,Palm Sunday_lower,Palm Sunday_upper,Republic Day,Republic Day_lower,Republic Day_upper,WEEKDAY,WEEKDAY_lower,WEEKDAY_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,end_of_month,end_of_month_lower,end_of_month_upper,extra_regressors_additive,extra_regressors_additive_lower,extra_regressors_additive_upper,holidays,holidays_lower,holidays_upper,hour,hour_lower,hour_upper,start_of_month,start_of_month_lower,start_of_month_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2018-07-01 00:00:00,5.129252,6.117940,39.271114,5.129252,5.129252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.266442,-0.266442,-0.266442,-2.339288,-2.339288,-2.339288,-2.386610,-2.386610,-2.386610,0.000000,0.000000,0.000000,0.076356,0.076356,0.076356,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.342797,0.342797,0.342797,-0.029033,-0.029033,-0.029033,0.0,0.0,0.0,16.280429
1,2018-07-01 01:00:00,5.129265,3.288786,22.909903,5.129265,5.129265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.266442,-0.266442,-0.266442,-2.968827,-2.968827,-2.968827,-2.995141,-2.995141,-2.995141,0.000000,0.000000,0.000000,0.071609,0.071609,0.071609,0.0,0.0,0.0,-0.004746,-0.004746,-0.004746,0.342797,0.342797,0.342797,-0.045295,-0.045295,-0.045295,0.0,0.0,0.0,8.674936
2,2018-07-01 02:00:00,5.129278,2.451740,15.418423,5.129278,5.129278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.266442,-0.266442,-0.266442,-3.337303,-3.337303,-3.337303,-3.342566,-3.342566,-3.342566,0.000000,0.000000,0.000000,0.066863,0.066863,0.066863,0.0,0.0,0.0,-0.009493,-0.009493,-0.009493,0.342797,0.342797,0.342797,-0.061600,-0.061600,-0.061600,0.0,0.0,0.0,6.001293
3,2018-07-01 03:00:00,5.129291,2.581059,15.264848,5.129291,5.129291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.266442,-0.266442,-0.266442,-3.302492,-3.302492,-3.302492,-3.286775,-3.286775,-3.286775,0.000000,0.000000,0.000000,0.062117,0.062117,0.062117,0.0,0.0,0.0,-0.014239,-0.014239,-0.014239,0.342797,0.342797,0.342797,-0.077834,-0.077834,-0.077834,0.0,0.0,0.0,6.213960
4,2018-07-01 04:00:00,5.129304,4.107513,27.722870,5.129304,5.129304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.266442,-0.266442,-0.266442,-2.816966,-2.816966,-2.816966,-2.780455,-2.780455,-2.780455,0.000000,0.000000,0.000000,0.057371,0.057371,0.057371,0.0,0.0,0.0,-0.018985,-0.018985,-0.018985,0.342797,0.342797,0.342797,-0.093882,-0.093882,-0.093882,0.0,0.0,0.0,10.097996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [23]:
forecast[['ds','yhat','yhat_lower','yhat_upper']]

,ds,yhat,yhat_lower,yhat_upper
0,2018-07-01 00:00:00,16.280429,6.117940,39.271114
1,2018-07-01 01:00:00,8.674936,3.288786,22.909903
2,2018-07-01 02:00:00,6.001293,2.451740,15.418423
3,2018-07-01 03:00:00,6.213960,2.581059,15.264848
4,2018-07-01 04:00:00,10.097996,4.107513,27.722870
...,...,...,...,...
739,2018-07-31 19:00:00,566.253960,227.737407,1375.010714
740,2018-07-31 20:00:00,303.750857,124.114611,851.503215
741,2018-07-31 21:00:00,143.565584,57.420512,362.061648
742,2018-07-31 22:00:00,65.341475,25.248837,159.775705


In [24]:
results=pd.DataFrame()
results['DateTime']=forecast['ds']
results['predicted']=forecast['yhat']
results

,DateTime,predicted
0,2018-07-01 00:00:00,16.280429
1,2018-07-01 01:00:00,8.674936
2,2018-07-01 02:00:00,6.001293
3,2018-07-01 03:00:00,6.213960
4,2018-07-01 04:00:00,10.097996
...,...,...
739,2018-07-31 19:00:00,566.253960
740,2018-07-31 20:00:00,303.750857
741,2018-07-31 21:00:00,143.565584
742,2018-07-31 22:00:00,65.341475


In [27]:
test['volume']=np.exp(test['volume'])
test

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,DateTime,volume,hour,end_of_month,start_of_month,WEEKDAY
10826,2018-07-01 00:00:00,50.0,0,0,1,6
10827,2018-07-01 01:00:00,29.0,1,0,1,6
10828,2018-07-01 02:00:00,4.0,2,0,1,6
10829,2018-07-01 03:00:00,2.0,3,0,1,6
10830,2018-07-01 04:00:00,22.0,4,0,1,6
...,...,...,...,...,...,...
11563,2018-07-31 19:00:00,545.0,19,1,0,1
11564,2018-07-31 20:00:00,874.0,20,1,0,1
11565,2018-07-31 21:00:00,386.0,21,1,0,1
11566,2018-07-31 22:00:00,223.0,22,1,0,1


In [28]:
import plotly
import plotly.graph_objects as go

# Create random data with numpy
fig = go.Figure()
trace1=go.Scatter(x=results['DateTime'], y=results['predicted'],
                     mode='lines',
                     name='predicted')
trace2=go.Scatter(x=test['DateTime'], y=test['volume'],
                    mode='lines',
                    name='actual')

data=[trace2,trace1]


updatemenus = list([
    dict(active=1,
         buttons=list([
            dict(label='Log Scale',
                 method='update',
                 args=[{'visible': [True, True]},
                       {'title': 'Log scale',
                        'yaxis': {'type': 'log'}}]),
            dict(label='Linear Scale',
                 method='update',
                 args=[{'visible': [True, True]},
                       {'title': 'Linear scale',
                        'yaxis': {'type': 'linear'}}])
            ]),
        )
    ])

layout = dict(updatemenus=updatemenus, title='Linear scale')
fig = go.Figure(data=data, layout=layout)
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(title_text="LSTM UNIVARIATE MODEL")


plotly.offline.iplot(fig)